In [1]:
import easyocr

from PIL import Image
import pytesseract

import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt

import numpy as np
import Levenshtein as lev

from enum import IntEnum

import cv2
import os
import optuna

C:\ProgramData\Anaconda3\envs\ocr\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

import json

In [3]:

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [4]:
data_dir = r'E:\datasets\MADE\3_graduation\parthplc\archive\data\\'

train_path = data_dir + 'train.jsonl'
dev_path = data_dir + 'dev.jsonl'

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(dev_path, lines=True)

In [5]:
train_data['path'] = train_data['img'].apply(lambda x: os.path.join(data_dir, x))
train_data

,id,img,label,text,path
0,42953,img/42953.png,0,its their character not their color that matters,E:\datasets\MADE\3_graduation\parthplc\archive...
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,E:\datasets\MADE\3_graduation\parthplc\archive...
2,13894,img/13894.png,0,putting bows on your pet,E:\datasets\MADE\3_graduation\parthplc\archive...
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,E:\datasets\MADE\3_graduation\parthplc\archive...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",E:\datasets\MADE\3_graduation\parthplc\archive...
...,...,...,...,...,...
8495,10423,img/10423.png,1,nobody wants to hang auschwitz me,E:\datasets\MADE\3_graduation\parthplc\archive...
8496,98203,img/98203.png,1,when god grants you a child after 20 years of ...,E:\datasets\MADE\3_graduation\parthplc\archive...
8497,36947,img/36947.png,1,gays on social media: equality! body positivit...,E:\datasets\MADE\3_graduation\parthplc\archive...
8498,16492,img/16492.png,1,having a bad day? you could be a siamese twin ...,E:\datasets\MADE\3_graduation\parthplc\archive...


In [6]:
test_data['path'] = test_data['img'].apply(lambda x: os.path.join(data_dir, x))
test_data

,id,img,label,text,path
0,8291,img/08291.png,1,white people is this a shooting range,E:\datasets\MADE\3_graduation\parthplc\archive...
1,46971,img/46971.png,1,bravery at its finest,E:\datasets\MADE\3_graduation\parthplc\archive...
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,E:\datasets\MADE\3_graduation\parthplc\archive...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,E:\datasets\MADE\3_graduation\parthplc\archive...
4,80243,img/80243.png,1,mississippi wind chime,E:\datasets\MADE\3_graduation\parthplc\archive...
...,...,...,...,...,...
495,83675,img/83675.png,0,i'm gonna be like phelps one day,E:\datasets\MADE\3_graduation\parthplc\archive...
496,37198,img/37198.png,0,when you're so relaxed you can feel yourself g...,E:\datasets\MADE\3_graduation\parthplc\archive...
497,48670,img/48670.png,0,look at this sandwich maker club i found on wi...,E:\datasets\MADE\3_graduation\parthplc\archive...
498,9863,img/09863.png,0,diverse group of women,E:\datasets\MADE\3_graduation\parthplc\archive...


In [7]:
easyocr_reader = easyocr.Reader(['ru','en']) # this needs to run only once to load the model into memory

In [8]:
class ImageTransformType(IntEnum):
    No = 0
    Gray = 1
    Binary = 2
    BinaryInv = 3
    AdaptiveMean = 4
    AdaptiveMeanInv = 5
    AdaptiveGaussian = 6
    AdaptiveGaussianInv = 7

In [9]:
def recognize(path:list, recognizer_name: str, image_transform: ImageTransformType, thresh: int, maxval: int, 
              block_size:int, c:float) -> list:
    recognized_text = []    
    for i in range(len(path)):
        #print(path[i])
        image = cv2.imread(path[i], cv2.COLOR_BGR2RGB)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        if image_transform == ImageTransformType.Gray:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
        if image_transform == ImageTransformType.Binary:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY)
            
        if image_transform == ImageTransformType.BinaryInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY_INV)

        if image_transform == ImageTransformType.AdaptiveMean:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c)

        if image_transform == ImageTransformType.AdaptiveMeanInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussian:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussianInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if recognizer_name == 'easyocr':            
            recognized_text.append(' '.join(easyocr_reader.readtext(image, detail=0)))
            
        elif recognizer_name == 'pytesseract':
            recognized_text.append(pytesseract.image_to_string(image).replace('\n', ' '))
            
    return recognized_text


def loss(data:list, text:list, recognizer_name: str, image_transform: ImageTransformType, thresh: int, maxval: int, block_size:int, c:float):
    recognized_text = recognize(data, recognizer_name, image_transform, thresh, maxval, block_size, c)
    try:
        distances = []
        for i in range(len(data)):
            txt = text[i]
            distances.append(lev.distance(txt, recognized_text[i]) / len(text))

        distances_mean = np.mean(distances)
    except Exception as ex:
        return 100500
    return distances_mean

In [11]:
loss(test_data['path'].values, 
     test_data['text'].values,
     recognizer_name='easyocr',
     image_transform=ImageTransformType.No,
     thresh=0,
     maxval=0,
     block_size=0,
     c=0)

0.058572000000000006

In [12]:
loss(test_data['path'].values, 
     test_data['text'].values,
     recognizer_name='pytesseract',
     image_transform=ImageTransformType.No,
     thresh=0,
     maxval=0,
     block_size=0,
     c=0)

0.06989200000000001

In [14]:
def objective(trial):
    #recognizer_name = trial.suggest_categorical("recognizer_name", ['easyocr', 'pytesseract'])
    image_transform = trial.suggest_categorical("image_transform", [ImageTransformType.No,
                                                                    ImageTransformType.Gray,
                                                                    ImageTransformType.Binary,
                                                                    ImageTransformType.BinaryInv])
    thresh = trial.suggest_int("thresh", 0, 256)
    maxval = trial.suggest_int("maxval", 0, 256)
    
    return loss(test_data['path'].values, 
     test_data['text'].values,
                recognizer_name='easyocr',#recognizer_name,
                image_transform=image_transform,
                thresh=thresh,
                maxval=maxval,
                block_size=0,
                c=0)

study = optuna.create_study()
study.optimize(objective, n_trials=300)


[I 2022-10-18 15:37:39,164] A new study created in memory with name: no-name-892573f1-731f-4797-b77c-132bbdea5e0a
[I 2022-10-18 15:38:25,977] Trial 0 finished with value: 0.0463 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 223, 'maxval': 113}. Best is trial 0 with value: 0.0463.
[I 2022-10-18 15:39:15,128] Trial 1 finished with value: 0.064952 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 145, 'maxval': 214}. Best is trial 0 with value: 0.0463.
[I 2022-10-18 15:40:05,174] Trial 2 finished with value: 0.058912000000000006 and parameters: {'image_transform': <ImageTransformType.Gray: 1>, 'thresh': 83, 'maxval': 21}. Best is trial 0 with value: 0.0463.
[I 2022-10-18 15:40:58,516] Trial 3 finished with value: 0.084512 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 71, 'maxval': 110}. Best is trial 0 with value: 0.0463.
[I 2022-10-18 15:41:49,310] Trial 4 finished with value: 0.058912000000

[I 2022-10-18 16:08:30,919] Trial 36 finished with value: 0.04701600000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 232, 'maxval': 247}. Best is trial 30 with value: 0.04416400000000001.
[I 2022-10-18 16:08:53,942] Trial 37 finished with value: 0.10858 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 149, 'maxval': 0}. Best is trial 30 with value: 0.04416400000000001.
[I 2022-10-18 16:09:39,399] Trial 38 finished with value: 0.042852 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 240, 'maxval': 114}. Best is trial 38 with value: 0.042852.
[I 2022-10-18 16:10:26,631] Trial 39 finished with value: 0.05612400000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 182, 'maxval': 119}. Best is trial 38 with value: 0.042852.
[I 2022-10-18 16:11:13,425] Trial 40 finished with value: 0.05052 and parameters: {'image_transform': <ImageTransformType.Binar

[I 2022-10-18 16:36:49,311] Trial 73 finished with value: 0.045048000000000005 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 228, 'maxval': 90}. Best is trial 72 with value: 0.041476000000000006.
[I 2022-10-18 16:37:35,639] Trial 74 finished with value: 0.041704000000000005 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 250, 'maxval': 81}. Best is trial 72 with value: 0.041476000000000006.
[I 2022-10-18 16:38:21,946] Trial 75 finished with value: 0.04117200000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 80}. Best is trial 75 with value: 0.04117200000000001.
[I 2022-10-18 16:38:45,144] Trial 76 finished with value: 0.10858 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 255, 'maxval': 82}. Best is trial 75 with value: 0.04117200000000001.
[I 2022-10-18 16:39:34,863] Trial 77 finished with value: 0.058912000000000006 and parameter

[I 2022-10-18 17:03:49,343] Trial 108 finished with value: 0.058572000000000006 and parameters: {'image_transform': <ImageTransformType.No: 0>, 'thresh': 235, 'maxval': 71}. Best is trial 78 with value: 0.040928000000000006.
[I 2022-10-18 17:04:35,777] Trial 109 finished with value: 0.041376 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 250, 'maxval': 92}. Best is trial 78 with value: 0.040928000000000006.
[I 2022-10-18 17:05:22,815] Trial 110 finished with value: 0.049516000000000004 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 222, 'maxval': 61}. Best is trial 78 with value: 0.040928000000000006.
[I 2022-10-18 17:06:08,029] Trial 111 finished with value: 0.04138800000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 250, 'maxval': 93}. Best is trial 78 with value: 0.040928000000000006.
[I 2022-10-18 17:06:53,407] Trial 112 finished with value: 0.043328000000000005 and paramete

[I 2022-10-18 17:29:33,104] Trial 143 finished with value: 0.04096000000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 251, 'maxval': 92}. Best is trial 78 with value: 0.040928000000000006.
[I 2022-10-18 17:29:56,261] Trial 144 finished with value: 0.10858 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 256, 'maxval': 82}. Best is trial 78 with value: 0.040928000000000006.
[I 2022-10-18 17:30:40,871] Trial 145 finished with value: 0.04275200000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 240, 'maxval': 92}. Best is trial 78 with value: 0.040928000000000006.
[I 2022-10-18 17:31:25,103] Trial 146 finished with value: 0.040944 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 105}. Best is trial 78 with value: 0.040928000000000006.
[I 2022-10-18 17:32:09,012] Trial 147 finished with value: 0.040944 and parameters: {'image_transf

[I 2022-10-18 17:54:00,104] Trial 178 finished with value: 0.041384 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 78}. Best is trial 156 with value: 0.040752000000000003.
[I 2022-10-18 17:54:23,480] Trial 179 finished with value: 0.10858 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 256, 'maxval': 87}. Best is trial 156 with value: 0.040752000000000003.
[I 2022-10-18 17:55:08,503] Trial 180 finished with value: 0.045320000000000006 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 235, 'maxval': 72}. Best is trial 156 with value: 0.040752000000000003.
[I 2022-10-18 17:55:53,173] Trial 181 finished with value: 0.040968000000000004 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 251, 'maxval': 93}. Best is trial 156 with value: 0.040752000000000003.
[I 2022-10-18 17:56:37,752] Trial 182 finished with value: 0.04138 and parameters: {'image_tr

[I 2022-10-18 18:21:22,349] Trial 213 finished with value: 0.10858 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 256, 'maxval': 87}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 18:22:17,307] Trial 214 finished with value: 0.041136000000000006 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 247, 'maxval': 97}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 18:23:11,834] Trial 215 finished with value: 0.04100800000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 106}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 18:24:05,407] Trial 216 finished with value: 0.042456 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 242, 'maxval': 92}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 18:24:58,985] Trial 217 finished with value: 0.041648000000000004 and parameters: {

[I 2022-10-18 18:50:09,063] Trial 248 finished with value: 0.041264 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 247, 'maxval': 101}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 18:51:18,745] Trial 249 finished with value: 0.08218 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 251, 'maxval': 15}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 18:52:16,179] Trial 250 finished with value: 0.058572000000000006 and parameters: {'image_transform': <ImageTransformType.No: 0>, 'thresh': 242, 'maxval': 96}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 18:53:09,610] Trial 251 finished with value: 0.041044000000000004 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 82}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 18:54:10,942] Trial 252 finished with value: 0.08776400000000001 and parameters: {'image_

[I 2022-10-18 19:20:38,670] Trial 283 finished with value: 0.041468000000000005 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 247, 'maxval': 84}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 19:21:23,903] Trial 284 finished with value: 0.04093200000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 251, 'maxval': 96}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 19:22:08,291] Trial 285 finished with value: 0.04304800000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 239, 'maxval': 103}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 19:22:52,238] Trial 286 finished with value: 0.041428000000000006 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 248, 'maxval': 97}. Best is trial 194 with value: 0.04052800000000001.
[I 2022-10-18 19:23:15,540] Trial 287 finished with value: 0.10858 and pa

In [23]:
print("Value:", study.best_value)
print("Parameters:", study.best_params)

Value: 0.04052800000000001
Parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 253, 'maxval': 95}


In [27]:
def objective(trial):
    #recognizer_name = trial.suggest_categorical("recognizer_name", ['easyocr', 'pytesseract'])
    image_transform = trial.suggest_categorical("image_transform", [ImageTransformType.AdaptiveMean,
                                                                   ImageTransformType.AdaptiveMeanInv,
                                                                   ImageTransformType.AdaptiveGaussian,
                                                                   ImageTransformType.AdaptiveGaussianInv])
    thresh = trial.suggest_int("thresh", 0, 256)
    maxval = trial.suggest_int("maxval", 0, 256)
    block_size = trial.suggest_int("block_size", 3, 21, 2)
    c = trial.suggest_int("c", -10, 10)
 
    return loss(test_data['path'].values, 
     test_data['text'].values,
                recognizer_name='easyocr',#recognizer_name,
                image_transform=image_transform,
                thresh=thresh,
                maxval=maxval,
                block_size=block_size,
                c=c)

study = optuna.create_study()
study.optimize(objective, n_trials=300)


[I 2022-10-19 06:30:09,449] A new study created in memory with name: no-name-5b571857-245d-4254-99d3-2a9fa1c8ad7b
[I 2022-10-19 06:30:40,448] Trial 0 finished with value: 0.10616 and parameters: {'image_transform': <ImageTransformType.AdaptiveMeanInv: 5>, 'thresh': 169, 'maxval': 152, 'block_size': 3, 'c': 0}. Best is trial 0 with value: 0.10616.
[I 2022-10-19 06:31:27,899] Trial 1 finished with value: 0.09406400000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMeanInv: 5>, 'thresh': 5, 'maxval': 59, 'block_size': 9, 'c': 8}. Best is trial 1 with value: 0.09406400000000001.
[I 2022-10-19 06:32:14,405] Trial 2 finished with value: 0.09172 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 11, 'maxval': 179, 'block_size': 7, 'c': 6}. Best is trial 2 with value: 0.09172.
[I 2022-10-19 06:33:00,633] Trial 3 finished with value: 0.104696 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 177, 'm

[I 2022-10-19 06:57:53,258] Trial 31 finished with value: 0.064928 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 74, 'maxval': 160, 'block_size': 15, 'c': -10}. Best is trial 22 with value: 0.064608.
[I 2022-10-19 06:58:47,723] Trial 32 finished with value: 0.06224000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 76, 'maxval': 143, 'block_size': 17, 'c': -10}. Best is trial 32 with value: 0.06224000000000001.
[I 2022-10-19 06:59:43,061] Trial 33 finished with value: 0.06332 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 15, 'maxval': 141, 'block_size': 17, 'c': -9}. Best is trial 32 with value: 0.06224000000000001.
[I 2022-10-19 07:00:39,163] Trial 34 finished with value: 0.063996 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 1, 'maxval': 109, 'block_size': 19, 'c': -6}. Best is trial 32 with value: 0.06224000000000001.
[I 2022-

[I 2022-10-19 07:24:06,063] Trial 63 finished with value: 0.06971600000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 34, 'maxval': 46, 'block_size': 21, 'c': -7}. Best is trial 51 with value: 0.062224.
[I 2022-10-19 07:24:54,226] Trial 64 finished with value: 0.06734400000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 65, 'maxval': 58, 'block_size': 19, 'c': -6}. Best is trial 51 with value: 0.062224.
[I 2022-10-19 07:25:47,312] Trial 65 finished with value: 0.08274800000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 26, 'maxval': 28, 'block_size': 19, 'c': -4}. Best is trial 51 with value: 0.062224.
[I 2022-10-19 07:26:35,029] Trial 66 finished with value: 0.06242000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 90, 'maxval': 72, 'block_size': 21, 'c': -8}. Best is trial 51 with value: 0.062224.
[I 2022-10-1

[I 2022-10-19 07:48:39,844] Trial 94 finished with value: 0.06116800000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 139, 'maxval': 131, 'block_size': 19, 'c': -10}. Best is trial 72 with value: 0.06047200000000001.
[I 2022-10-19 07:49:26,660] Trial 95 finished with value: 0.06949600000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveMeanInv: 5>, 'thresh': 135, 'maxval': 140, 'block_size': 19, 'c': -10}. Best is trial 72 with value: 0.06047200000000001.
[I 2022-10-19 07:50:12,933] Trial 96 finished with value: 0.064296 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 141, 'maxval': 133, 'block_size': 17, 'c': -8}. Best is trial 72 with value: 0.06047200000000001.
[I 2022-10-19 07:50:58,923] Trial 97 finished with value: 0.062616 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 156, 'maxval': 174, 'block_size': 19, 'c': -10}. Best is trial 72 with valu

[I 2022-10-19 08:12:46,851] Trial 125 finished with value: 0.05962800000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 113, 'maxval': 88, 'block_size': 21, 'c': -10}. Best is trial 116 with value: 0.05937200000000001.
[I 2022-10-19 08:13:33,792] Trial 126 finished with value: 0.061936000000000005 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 100, 'maxval': 78, 'block_size': 21, 'c': -10}. Best is trial 116 with value: 0.05937200000000001.
[I 2022-10-19 08:14:20,183] Trial 127 finished with value: 0.060888000000000005 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 113, 'maxval': 69, 'block_size': 21, 'c': -10}. Best is trial 116 with value: 0.05937200000000001.
[I 2022-10-19 08:15:07,001] Trial 128 finished with value: 0.061248000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 114, 'maxval': 66, 'block_size': 21, 'c': -1

[I 2022-10-19 08:36:31,378] Trial 155 finished with value: 0.06028000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 105, 'maxval': 95, 'block_size': 21, 'c': -9}. Best is trial 116 with value: 0.05937200000000001.
[I 2022-10-19 08:37:18,593] Trial 156 finished with value: 0.059568 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 117, 'maxval': 101, 'block_size': 21, 'c': -10}. Best is trial 116 with value: 0.05937200000000001.
[I 2022-10-19 08:38:06,631] Trial 157 finished with value: 0.062256000000000006 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 117, 'maxval': 99, 'block_size': 21, 'c': -9}. Best is trial 116 with value: 0.05937200000000001.
[I 2022-10-19 08:38:54,572] Trial 158 finished with value: 0.059480000000000005 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 95, 'maxval': 109, 'block_size': 21, 'c': -10}. Best is t

[I 2022-10-19 09:01:04,710] Trial 186 finished with value: 0.08676000000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveMeanInv: 5>, 'thresh': 107, 'maxval': 94, 'block_size': 7, 'c': -10}. Best is trial 116 with value: 0.05937200000000001.
[I 2022-10-19 09:01:51,848] Trial 187 finished with value: 0.059716 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 97, 'maxval': 103, 'block_size': 21, 'c': -10}. Best is trial 116 with value: 0.05937200000000001.
[I 2022-10-19 09:02:39,502] Trial 188 finished with value: 0.06038 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 103, 'maxval': 100, 'block_size': 21, 'c': -9}. Best is trial 116 with value: 0.05937200000000001.
[I 2022-10-19 09:03:26,407] Trial 189 finished with value: 0.05938000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 74, 'maxval': 94, 'block_size': 21, 'c': -10}. Best is trial 116 with va

[I 2022-10-19 09:26:19,370] Trial 218 finished with value: 0.060888000000000005 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 76, 'maxval': 77, 'block_size': 21, 'c': -9}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 09:27:07,590] Trial 219 finished with value: 0.06033600000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 65, 'maxval': 94, 'block_size': 21, 'c': -9}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 09:27:55,142] Trial 220 finished with value: 0.05955200000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 84, 'maxval': 110, 'block_size': 21, 'c': -10}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 09:28:42,425] Trial 221 finished with value: 0.059488 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 69, 'maxval': 93, 'block_size': 21, 'c': -10}. Best is trial 190 with value: 0.059356.
[I 2022-10-

[I 2022-10-19 09:51:52,127] Trial 250 finished with value: 0.059640000000000006 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 87, 'maxval': 105, 'block_size': 21, 'c': -10}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 09:52:39,818] Trial 251 finished with value: 0.059716 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 77, 'maxval': 103, 'block_size': 21, 'c': -10}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 09:53:27,803] Trial 252 finished with value: 0.060512 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 85, 'maxval': 101, 'block_size': 21, 'c': -9}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 09:54:17,035] Trial 253 finished with value: 0.07115600000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 82, 'maxval': 113, 'block_size': 21, 'c': 5}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 09:55:

[I 2022-10-19 10:17:25,043] Trial 282 finished with value: 0.05937200000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 78, 'maxval': 95, 'block_size': 21, 'c': -10}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 10:18:12,039] Trial 283 finished with value: 0.05938000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 77, 'maxval': 94, 'block_size': 21, 'c': -10}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 10:19:00,301] Trial 284 finished with value: 0.06020400000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 71, 'maxval': 84, 'block_size': 21, 'c': -9}. Best is trial 190 with value: 0.059356.
[I 2022-10-19 10:19:48,419] Trial 285 finished with value: 0.05938000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 77, 'maxval': 94, 'block_size': 21, 'c': -10}. Best is trial 190 with value: 0.059356.
[

In [28]:
print("Value:", study.best_value)
print("Parameters:", study.best_params)

Value: 0.059356
Parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 75, 'maxval': 96, 'block_size': 21, 'c': -10}


In [29]:
def objective(trial):
    #recognizer_name = trial.suggest_categorical("recognizer_name", ['easyocr', 'pytesseract'])
    image_transform = trial.suggest_categorical("image_transform", [ImageTransformType.No,
                                                                    ImageTransformType.Gray,
                                                                    ImageTransformType.Binary,
                                                                    ImageTransformType.BinaryInv])
    thresh = trial.suggest_int("thresh", 0, 256)
    maxval = trial.suggest_int("maxval", 0, 256)
    
    return loss(test_data['path'].values, 
     test_data['text'].values,
                recognizer_name='pytesseract',#recognizer_name,
                image_transform=image_transform,
                thresh=thresh,
                maxval=maxval,
                block_size=0,
                c=0)

study = optuna.create_study()
study.optimize(objective, n_trials=300)


[I 2022-10-19 10:31:12,483] A new study created in memory with name: no-name-4e08dd81-5b32-4156-8ef8-b8bbe9802950
[I 2022-10-19 10:32:37,450] Trial 0 finished with value: 0.068588 and parameters: {'image_transform': <ImageTransformType.Gray: 1>, 'thresh': 197, 'maxval': 222}. Best is trial 0 with value: 0.068588.
[I 2022-10-19 10:33:42,755] Trial 1 finished with value: 0.047776000000000006 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 210, 'maxval': 162}. Best is trial 1 with value: 0.047776000000000006.
[I 2022-10-19 10:35:40,093] Trial 2 finished with value: 0.06989200000000001 and parameters: {'image_transform': <ImageTransformType.No: 0>, 'thresh': 109, 'maxval': 237}. Best is trial 1 with value: 0.047776000000000006.
[I 2022-10-19 10:36:49,447] Trial 3 finished with value: 0.055876 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 191, 'maxval': 158}. Best is trial 1 with value: 0.047776000000000006.
[I 2022-10-19 10:3

[I 2022-10-19 11:15:50,627] Trial 35 finished with value: 0.046520000000000006 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 211, 'maxval': 180}. Best is trial 32 with value: 0.022656000000000003.
[I 2022-10-19 11:16:59,783] Trial 36 finished with value: 0.051616 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 196, 'maxval': 227}. Best is trial 32 with value: 0.022656000000000003.
[I 2022-10-19 11:18:07,678] Trial 37 finished with value: 0.020232000000000003 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 247, 'maxval': 156}. Best is trial 37 with value: 0.020232000000000003.
[I 2022-10-19 11:20:03,842] Trial 38 finished with value: 0.06989200000000001 and parameters: {'image_transform': <ImageTransformType.No: 0>, 'thresh': 217, 'maxval': 154}. Best is trial 37 with value: 0.020232000000000003.
[I 2022-10-19 11:21:25,861] Trial 39 finished with value: 0.068588 and parameters: {'image_transform'

[I 2022-10-19 11:57:49,756] Trial 71 finished with value: 0.018472000000000002 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 250, 'maxval': 71}. Best is trial 68 with value: 0.016440000000000003.
[I 2022-10-19 11:58:57,182] Trial 72 finished with value: 0.016424000000000005 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 252, 'maxval': 115}. Best is trial 72 with value: 0.016424000000000005.
[I 2022-10-19 12:00:06,049] Trial 73 finished with value: 0.03633200000000001 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 230, 'maxval': 138}. Best is trial 72 with value: 0.016424000000000005.
[I 2022-10-19 12:02:02,264] Trial 74 finished with value: 0.06989200000000001 and parameters: {'image_transform': <ImageTransformType.No: 0>, 'thresh': 239, 'maxval': 120}. Best is trial 72 with value: 0.016424000000000005.
[I 2022-10-19 12:03:09,555] Trial 75 finished with value: 0.015232000000000002 and paramete

[I 2022-10-19 12:35:33,800] Trial 106 finished with value: 0.016728 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 161}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 12:36:35,415] Trial 107 finished with value: 0.021764000000000002 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 246, 'maxval': 161}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 12:37:39,414] Trial 108 finished with value: 0.044964 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 214, 'maxval': 144}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 12:38:41,671] Trial 109 finished with value: 0.028952000000000006 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 238, 'maxval': 183}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 12:39:42,587] Trial 110 finished with value: 0.015632000000000004 and paramete

[I 2022-10-19 13:12:50,080] Trial 141 finished with value: 0.015696 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 253, 'maxval': 159}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 13:13:50,823] Trial 142 finished with value: 0.016820000000000005 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 167}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 13:14:53,040] Trial 143 finished with value: 0.025220000000000003 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 242, 'maxval': 176}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 13:15:53,930] Trial 144 finished with value: 0.01676 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 152}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 13:16:55,633] Trial 145 finished with value: 0.020664000000000002 and parameter

[I 2022-10-19 13:49:27,458] Trial 176 finished with value: 0.020544000000000003 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 247, 'maxval': 145}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 13:50:28,808] Trial 177 finished with value: 0.028940000000000004 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 238, 'maxval': 157}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 13:51:28,837] Trial 178 finished with value: 0.016880000000000003 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 165}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 13:52:29,892] Trial 179 finished with value: 0.025152 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 242, 'maxval': 149}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 13:53:31,845] Trial 180 finished with value: 0.034224000000000004 

[I 2022-10-19 14:24:20,290] Trial 210 finished with value: 0.016896 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 36}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 14:25:21,435] Trial 211 finished with value: 0.016804000000000003 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 252, 'maxval': 158}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 14:26:23,026] Trial 212 finished with value: 0.019656000000000007 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 248, 'maxval': 156}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 14:27:24,279] Trial 213 finished with value: 0.017864000000000005 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 251, 'maxval': 173}. Best is trial 75 with value: 0.015232000000000002.
[I 2022-10-19 14:28:26,254] Trial 214 finished with value: 0.023100000000000002 a

[I 2022-10-19 15:01:39,847] Trial 245 finished with value: 0.016456000000000002 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 252, 'maxval': 134}. Best is trial 221 with value: 0.015120000000000001.
[I 2022-10-19 15:02:48,388] Trial 246 finished with value: 0.016364 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 252, 'maxval': 132}. Best is trial 221 with value: 0.015120000000000001.
[I 2022-10-19 15:03:56,702] Trial 247 finished with value: 0.016344000000000004 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 252, 'maxval': 130}. Best is trial 221 with value: 0.015120000000000001.
[I 2022-10-19 15:05:05,942] Trial 248 finished with value: 0.024160000000000004 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 243, 'maxval': 130}. Best is trial 221 with value: 0.015120000000000001.
[I 2022-10-19 15:06:14,237] Trial 249 finished with value: 0.016492 and parameters: {'im

[I 2022-10-19 15:40:49,558] Trial 280 finished with value: 0.020272000000000002 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 247, 'maxval': 99}. Best is trial 221 with value: 0.015120000000000001.
[I 2022-10-19 15:42:12,381] Trial 281 finished with value: 0.068588 and parameters: {'image_transform': <ImageTransformType.Gray: 1>, 'thresh': 252, 'maxval': 241}. Best is trial 221 with value: 0.015120000000000001.
[I 2022-10-19 15:43:01,483] Trial 282 finished with value: 0.10858 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 256, 'maxval': 95}. Best is trial 221 with value: 0.015120000000000001.
[I 2022-10-19 15:44:10,692] Trial 283 finished with value: 0.030676000000000005 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 235, 'maxval': 111}. Best is trial 221 with value: 0.015120000000000001.
[I 2022-10-19 15:45:19,854] Trial 284 finished with value: 0.0241 and parameters: {'image_transform': <Im

In [30]:
print("Value:", study.best_value)
print("Parameters:", study.best_params)

Value: 0.015120000000000001
Parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 253, 'maxval': 158}


In [31]:
def objective(trial):
    #recognizer_name = trial.suggest_categorical("recognizer_name", ['easyocr', 'pytesseract'])
    image_transform = trial.suggest_categorical("image_transform", [ImageTransformType.AdaptiveMean,
                                                                   ImageTransformType.AdaptiveMeanInv,
                                                                   ImageTransformType.AdaptiveGaussian,
                                                                   ImageTransformType.AdaptiveGaussianInv])
    thresh = trial.suggest_int("thresh", 0, 256)
    maxval = trial.suggest_int("maxval", 0, 256)
    block_size = trial.suggest_int("block_size", 3, 21, 2)
    c = trial.suggest_int("c", -10, 10)
 
    return loss(test_data['path'].values, 
     test_data['text'].values,
                recognizer_name='pytesseract',#recognizer_name,
                image_transform=image_transform,
                thresh=thresh,
                maxval=maxval,
                block_size=block_size,
                c=c)

study = optuna.create_study()
study.optimize(objective, n_trials=300)


[I 2022-10-19 16:03:07,317] A new study created in memory with name: no-name-1adcadc1-0a00-41d4-a63a-6d8036581207
[I 2022-10-19 16:04:17,511] Trial 0 finished with value: 0.08933200000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 110, 'maxval': 109, 'block_size': 9, 'c': -7}. Best is trial 0 with value: 0.08933200000000001.
[I 2022-10-19 16:05:42,327] Trial 1 finished with value: 0.10555600000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 144, 'maxval': 41, 'block_size': 9, 'c': 0}. Best is trial 0 with value: 0.08933200000000001.
[I 2022-10-19 16:06:53,553] Trial 2 finished with value: 0.094504 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 173, 'maxval': 62, 'block_size': 7, 'c': 4}. Best is trial 0 with value: 0.08933200000000001.
[I 2022-10-19 16:08:03,338] Trial 3 finished with value: 0.09734000000000001 and parameters: {'image_transform': <Image

[I 2022-10-19 16:41:33,332] Trial 31 finished with value: 0.08184000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 39, 'maxval': 189, 'block_size': 7, 'c': -10}. Best is trial 16 with value: 0.081608.
[I 2022-10-19 16:42:43,349] Trial 32 finished with value: 0.08372800000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 21, 'maxval': 166, 'block_size': 5, 'c': -9}. Best is trial 16 with value: 0.081608.
[I 2022-10-19 16:43:53,407] Trial 33 finished with value: 0.08452000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 40, 'maxval': 230, 'block_size': 7, 'c': -8}. Best is trial 16 with value: 0.081608.
[I 2022-10-19 16:45:04,205] Trial 34 finished with value: 0.089188 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 136, 'maxval': 189, 'block_size': 9, 'c': -6}. Best is trial 16 with value: 0.081608.
[I 20

[I 2022-10-19 17:17:07,830] Trial 62 finished with value: 0.081232 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 111, 'maxval': 220, 'block_size': 3, 'c': -8}. Best is trial 61 with value: 0.07821600000000001.
[I 2022-10-19 17:18:15,638] Trial 63 finished with value: 0.088712 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 107, 'maxval': 220, 'block_size': 3, 'c': -4}. Best is trial 61 with value: 0.07821600000000001.
[I 2022-10-19 17:19:22,610] Trial 64 finished with value: 0.0821 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 145, 'maxval': 232, 'block_size': 3, 'c': -7}. Best is trial 61 with value: 0.07821600000000001.
[I 2022-10-19 17:20:29,337] Trial 65 finished with value: 0.081012 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 86, 'maxval': 211, 'block_size': 3, 'c': -8}. Best is trial 61 with value: 0.07821600000000

[I 2022-10-19 17:52:12,364] Trial 93 finished with value: 0.08196 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 65, 'maxval': 47, 'block_size': 3, 'c': -7}. Best is trial 61 with value: 0.07821600000000001.
[I 2022-10-19 17:53:18,615] Trial 94 finished with value: 0.07882800000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 125, 'maxval': 82, 'block_size': 3, 'c': -10}. Best is trial 61 with value: 0.07821600000000001.
[I 2022-10-19 17:54:25,923] Trial 95 finished with value: 0.081004 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 111, 'maxval': 112, 'block_size': 3, 'c': -8}. Best is trial 61 with value: 0.07821600000000001.
[I 2022-10-19 17:55:34,328] Trial 96 finished with value: 0.08357200000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 90, 'maxval': 93, 'block_size': 5, 'c': -10}. Best is trial 61 with va

[I 2022-10-19 18:27:02,785] Trial 124 finished with value: 0.07838400000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 134, 'maxval': 107, 'block_size': 3, 'c': -9}. Best is trial 122 with value: 0.078208.
[I 2022-10-19 18:28:09,626] Trial 125 finished with value: 0.078232 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 123, 'maxval': 83, 'block_size': 3, 'c': -9}. Best is trial 122 with value: 0.078208.
[I 2022-10-19 18:29:16,583] Trial 126 finished with value: 0.08099600000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 109, 'maxval': 84, 'block_size': 3, 'c': -8}. Best is trial 122 with value: 0.078208.
[I 2022-10-19 18:30:22,944] Trial 127 finished with value: 0.07886000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 57, 'maxval': 75, 'block_size': 3, 'c': -10}. Best is trial 122 with value: 0.078208

[I 2022-10-19 19:03:15,900] Trial 156 finished with value: 0.08566000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 118, 'maxval': 125, 'block_size': 3, 'c': 4}. Best is trial 122 with value: 0.078208.
[I 2022-10-19 19:04:24,898] Trial 157 finished with value: 0.082224 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 130, 'maxval': 94, 'block_size': 3, 'c': -7}. Best is trial 122 with value: 0.078208.
[I 2022-10-19 19:05:36,689] Trial 158 finished with value: 0.07908000000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 100, 'maxval': 112, 'block_size': 3, 'c': -10}. Best is trial 122 with value: 0.078208.
[I 2022-10-19 19:06:47,932] Trial 159 finished with value: 0.072524 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 122, 'maxval': 131, 'block_size': 3, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022

[I 2022-10-19 19:40:26,024] Trial 188 finished with value: 0.083636 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 117, 'maxval': 220, 'block_size': 5, 'c': -10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 19:41:33,864] Trial 189 finished with value: 0.07839600000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 122, 'maxval': 78, 'block_size': 3, 'c': -9}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 19:42:41,774] Trial 190 finished with value: 0.081184 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 79, 'maxval': 87, 'block_size': 3, 'c': -8}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 19:43:50,835] Trial 191 finished with value: 0.07827600000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 104, 'maxval': 97, 'block_size': 3, 'c': -9}. Best is trial 159 with value: 0.072524.
[I 2022-1

[I 2022-10-19 20:16:41,728] Trial 220 finished with value: 0.073112 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 81, 'maxval': 254, 'block_size': 3, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 20:17:49,844] Trial 221 finished with value: 0.07330800000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 83, 'maxval': 256, 'block_size': 3, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 20:18:58,070] Trial 222 finished with value: 0.07312 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 83, 'maxval': 253, 'block_size': 3, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 20:20:06,399] Trial 223 finished with value: 0.07317200000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 80, 'maxval': 252, 'block_size': 3, 'c': 10}. Best is trial 159 with value: 0.072524.

[I 2022-10-19 20:53:26,423] Trial 252 finished with value: 0.07330000000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 73, 'maxval': 246, 'block_size': 3, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 20:54:34,993] Trial 253 finished with value: 0.07316800000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 71, 'maxval': 247, 'block_size': 3, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 20:55:43,669] Trial 254 finished with value: 0.07330000000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 71, 'maxval': 246, 'block_size': 3, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 20:56:52,164] Trial 255 finished with value: 0.07316800000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 72, 'maxval': 247, 'block_size': 3, 'c': 10}. Best is trial 15

[I 2022-10-19 21:29:01,859] Trial 283 finished with value: 0.08274800000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 50, 'maxval': 228, 'block_size': 5, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 21:30:10,334] Trial 284 finished with value: 0.091632 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 63, 'maxval': 236, 'block_size': 9, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 21:31:19,273] Trial 285 finished with value: 0.07302000000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 41, 'maxval': 250, 'block_size': 3, 'c': 10}. Best is trial 159 with value: 0.072524.
[I 2022-10-19 21:32:27,713] Trial 286 finished with value: 0.07302000000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussianInv: 7>, 'thresh': 38, 'maxval': 250, 'block_size': 3, 'c': 10}. Best is trial 159 with valu

In [32]:
print("Value:", study.best_value)
print("Parameters:", study.best_params)

Value: 0.072524
Parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 122, 'maxval': 131, 'block_size': 3, 'c': 10}
